<a href="https://colab.research.google.com/github/pms-collab/quantum-dna-aligner/blob/main/ECG_%EA%B8%B0%EB%B0%98_%EB%B6%80%EC%A0%95%EB%A7%A5_%ED%8C%90%EB%8B%A8_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import json
import os

# 여기에 네 Kaggle 계정 정보 입력
KAGGLE_USERNAME = "remedeomnia"
KAGGLE_KEY = "KGAT_bf7924bc0d2f25565742751463a3132e"

kaggle_dict = {
    "username": KAGGLE_USERNAME,
    "key": KAGGLE_KEY,
}

# 1) 현재 작업 디렉토리에 kaggle.json 생성
with open("kaggle.json", "w") as f:
    json.dump(kaggle_dict, f)

# 2) ~/.kaggle 폴더로 이동 + 권한 설정
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
os.replace("kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

print("kaggle.json created at ~/.kaggle/kaggle.json")


kaggle.json created at ~/.kaggle/kaggle.json


In [16]:
!kaggle --version


Kaggle API 1.7.4.5


In [17]:
# ECG Heartbeat Categorization Dataset 다운로드
!kaggle datasets download -d shayanfazeli/heartbeat

# 압축 해제
!unzip -o heartbeat.zip


Dataset URL: https://www.kaggle.com/datasets/shayanfazeli/heartbeat
License(s): unknown
  0% 0.00/98.8M [00:00<?, ?B/s]
100% 98.8M/98.8M [00:00<00:00, 1.46GB/s]
Archive:  heartbeat.zip
  inflating: mitbih_test.csv         
  inflating: mitbih_train.csv        
  inflating: ptbdb_abnormal.csv      
  inflating: ptbdb_normal.csv        


In [19]:
import pandas as pd

df_train = pd.read_csv("mitbih_train.csv", header=None)
df_test  = pd.read_csv("mitbih_test.csv", header=None)

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

X = df_train.iloc[:, :-1].values  # (N, 187)
y = df_train.iloc[:, -1].values.astype(int)  # (N,)

print("Feature shape:", X.shape)
print("Label shape:", y.shape)
print("Label distribution:\n", pd.Series(y).value_counts().sort_index())

Train shape: (87554, 188)
Test shape: (21892, 188)
Feature shape: (87554, 187)
Label shape: (87554,)
Label distribution:
 0    72471
1     2223
2     5788
3      641
4     6431
Name: count, dtype: int64


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score

# train / validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape[0], "Val size:", X_val.shape[0])

baseline_clf = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            multi_class="multinomial",
            solver="lbfgs",
            max_iter=1000,
            class_weight="balanced",
            n_jobs=-1,
        )),
    ]
)

print("\nTraining baseline Logistic Regression...")
baseline_clf.fit(X_train, y_train)

y_val_pred = baseline_clf.predict(X_val)

acc = accuracy_score(y_val, y_val_pred)
macro_f1 = f1_score(y_val, y_val_pred, average="macro")

print(f"\n[Baseline] accuracy: {acc:.4f}")
print(f"[Baseline] macro F1: {macro_f1:.4f}\n")

print("Classification report (baseline):")
print(classification_report(y_val, y_val_pred, digits=4))


Train size: 70043 Val size: 17511

Training baseline Logistic Regression...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



[Baseline] accuracy: 0.6747
[Baseline] macro F1: 0.4822

Classification report (baseline):
              precision    recall  f1-score   support

           0     0.9735    0.6443    0.7754     14494
           1     0.1518    0.6899    0.2488       445
           2     0.2981    0.7401    0.4250      1158
           3     0.0815    0.8594    0.1488       128
           4     0.7193    0.9347    0.8130      1286

    accuracy                         0.6747     17511
   macro avg     0.4448    0.7737    0.4822     17511
weighted avg     0.8828    0.6747    0.7370     17511



In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight

print("TensorFlow version:", tf.__version__)

# CNN 입력: (N, 187, 1)
X_train_cnn = X_train[..., np.newaxis]
X_val_cnn   = X_val[..., np.newaxis]

print("X_train_cnn shape:", X_train_cnn.shape)
print("X_val_cnn shape:", X_val_cnn.shape)

# class weight 계산
classes = np.unique(y_train)
class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)
class_weights = dict(zip(classes, class_weights_array))
print("Class weights:", class_weights)

def build_cnn_model(input_length=187, n_classes=5):
    model = keras.Sequential([
        layers.Conv1D(32, 5, padding="same", activation="relu", input_shape=(input_length, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),

        layers.Conv1D(64, 5, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),

        layers.Conv1D(128, 3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),

        layers.Dense(64, activation="relu"),
        layers.Dropout(0.5),

        layers.Dense(n_classes, activation="softmax"),
    ])
    return model

model = build_cnn_model()

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

EPOCHS = 15
BATCH_SIZE = 256

history = model.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weights,
    verbose=1,
)


TensorFlow version: 2.19.0
X_train_cnn shape: (70043, 187, 1)
X_val_cnn shape: (17511, 187, 1)
Class weights: {np.int64(0): np.float64(0.2416234023837039), np.int64(1): np.float64(7.878852643419573), np.int64(2): np.float64(3.0256155507559397), np.int64(3): np.float64(27.307212475633527), np.int64(4): np.float64(2.722759961127308)}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 187, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 187, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 93, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 93, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 93, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 46, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 46, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 46, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,677 (174.52 KB)

 Trainable params: 44,229 (172.77 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 74s 259ms/step - accuracy: 0.3905 - loss: 1.0142 - val_accuracy: 0.0661 - val_loss: 3.0412
Epoch 2/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 57s 209ms/step - accuracy: 0.7523 - loss: 0.4958 - val_accuracy: 0.0859 - val_loss: 3.9106
Epoch 3/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 77s 192ms/step - accuracy: 0.8188 - loss: 0.4093 - val_accuracy: 0.8044 - val_loss: 0.5549
Epoch 4/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 83s 196ms/step - accuracy: 0.8388 - loss: 0.3573 - val_accuracy: 0.9385 - val_loss: 0.2658
Epoch 5/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 81s 191ms/step - accuracy: 0.8602 - loss: 0.3298 - val_accuracy: 0.7270 - val_loss: 0.7342
Epoch 6/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 85s 201ms/step - accuracy: 0.8678 - loss: 0.2979 - val_accuracy: 0.9144 - val_loss: 0.2826
Epoch 7/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 52s 190ms/step - accuracy: 0.8611 - loss: 0.3077 - val_accuracy: 0.8594 - val_loss: 0.4370
Epoch 8/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 51s 187ms/step - accuracy: 0.8830 - loss: 0

In [32]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

y_val_prob_cnn = model.predict(X_val_cnn)
y_val_pred_cnn = np.argmax(y_val_prob_cnn, axis=1)

acc_cnn = accuracy_score(y_val, y_val_pred_cnn)
macro_f1_cnn = f1_score(y_val, y_val_pred_cnn, average="macro")

print(f"\n[CNN v1] accuracy: {acc_cnn:.4f}")
print(f"[CNN v1] macro F1: {macro_f1_cnn:.4f}\n")

print("Classification report (CNN v1):")
print(classification_report(y_val, y_val_pred_cnn, digits=4))


548/548 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step

[CNN v1] accuracy: 0.9462
[CNN v1] macro F1: 0.8266

Classification report (CNN v1):
              precision    recall  f1-score   support

           0     0.9866    0.9545    0.9703     14494
           1     0.4181    0.7910    0.5470       445
           2     0.9324    0.8817    0.9063      1158
           3     0.8738    0.7031    0.7792       128
           4     0.8778    0.9891    0.9302      1286

    accuracy                         0.9462     17511
   macro avg     0.8177    0.8639    0.8266     17511
weighted avg     0.9598    0.9462    0.9509     17511



In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight

print("TensorFlow version:", tf.__version__)

# CNN 입력: (N, 187, 1)
X_train_cnn = X_train[..., np.newaxis]
X_val_cnn   = X_val[..., np.newaxis]

print("X_train_cnn shape:", X_train_cnn.shape)
print("X_val_cnn shape:", X_val_cnn.shape)

# class weight 계산
classes = np.unique(y_train)
class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)
class_weights = dict(zip(classes, class_weights_array))
# v2용 class weight 조정
class_weights_v2 = class_weights.copy()
class_weights_v2[3] = class_weights_v2[3] * 2.0  # Class 3를 더 중요하게

print("Class weights (v2):", class_weights_v2)

def build_cnn_model(input_length=187, n_classes=5):
    model = keras.Sequential([
        layers.Conv1D(32, 5, padding="same", activation="relu", input_shape=(input_length, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),

        layers.Conv1D(64, 5, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),

        layers.Conv1D(128, 3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling1D(),

        layers.Dense(64, activation="relu"),
        layers.Dropout(0.5),

        layers.Dense(n_classes, activation="softmax"),
    ])
    return model

model = build_cnn_model()

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

EPOCHS = 15
BATCH_SIZE = 256

history_v2 = model.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=EPOCHS,  # 예: 15 또는 20
    batch_size=BATCH_SIZE,
    class_weight=class_weights_v2,
    verbose=1,
)

TensorFlow version: 2.19.0
X_train_cnn shape: (70043, 187, 1)
X_val_cnn shape: (17511, 187, 1)
Class weights (v2): {np.int64(0): np.float64(0.2416234023837039), np.int64(1): np.float64(7.878852643419573), np.int64(2): np.float64(3.0256155507559397), np.int64(3): np.float64(54.614424951267054), np.int64(4): np.float64(2.722759961127308)}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 187, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 187, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 93, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 93, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 93, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 46, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 46, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 46, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_5      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,677 (174.52 KB)

 Trainable params: 44,229 (172.77 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 61s 211ms/step - accuracy: 0.3418 - loss: 1.2524 - val_accuracy: 0.0661 - val_loss: 4.2180
Epoch 2/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 79s 202ms/step - accuracy: 0.6332 - loss: 0.6421 - val_accuracy: 0.0812 - val_loss: 3.4185
Epoch 3/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 54s 197ms/step - accuracy: 0.7331 - loss: 0.5354 - val_accuracy: 0.8621 - val_loss: 0.4651
Epoch 4/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 82s 196ms/step - accuracy: 0.7727 - loss: 0.5097 - val_accuracy: 0.8979 - val_loss: 0.3670
Epoch 5/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.8212 - loss: 0.4164 - val_accuracy: 0.6723 - val_loss: 0.9450
Epoch 6/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 54s 195ms/step - accuracy: 0.7958 - loss: 0.4514 - val_accuracy: 0.4332 - val_loss: 1.6715
Epoch 7/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 54s 196ms/step - accuracy: 0.8503 - loss: 0.3857 - val_accuracy: 0.8753 - val_loss: 0.4160
Epoch 8/15
274/274 ━━━━━━━━━━━━━━━━━━━━ 54s 196ms/step - accuracy: 0.8450 - loss: 0

In [34]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    recall_score,
)
import numpy as np

# ===== 1. 예측 =====
# softmax 확률
y_val_prob_v2 = model.predict(X_val_cnn)
# 가장 확률 높은 클래스 인덱스
y_val_pred_v2 = np.argmax(y_val_prob_v2, axis=1)

# ===== 2. 전체 accuracy / macro F1 =====
acc_v2 = accuracy_score(y_val, y_val_pred_v2)
macro_f1_v2 = f1_score(y_val, y_val_pred_v2, average="macro")

print(f"[CNN v2] accuracy: {acc_v2:.4f}")
print(f"[CNN v2] macro F1: {macro_f1_v2:.4f}\n")

# ===== 3. 클래스별 precision / recall / F1 =====
print("Classification report (CNN v2):")
print(classification_report(y_val, y_val_pred_v2, digits=4))

# ===== 4. rare-class (1–4) recall 평균 =====
rare_labels = [1, 2, 3, 4]
rare_recalls_v2 = []

print("\nRare-class recall (CNN v2):")
for lbl in rare_labels:
    r = recall_score(
        y_val,
        y_val_pred_v2,
        labels=[lbl],
        average="macro",
        zero_division=0,
    )
    rare_recalls_v2.append(r)
    print(f"  Class {lbl} recall (v2): {r:.4f}")

print(f"Rare-class mean recall (v2): {np.mean(rare_recalls_v2):.4f}")


548/548 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step
[CNN v2] accuracy: 0.7989
[CNN v2] macro F1: 0.6564

Classification report (CNN v2):
              precision    recall  f1-score   support

           0     0.9918    0.7760    0.8707     14494
           1     0.1259    0.8607    0.2196       445
           2     0.6671    0.9603    0.7873      1158
           3     0.3290    0.7891    0.4644       128
           4     0.9939    0.8919    0.9402      1286

    accuracy                         0.7989     17511
   macro avg     0.6215    0.8556    0.6564     17511
weighted avg     0.9436    0.7989    0.8508     17511


Rare-class recall (CNN v2):
  Class 1 recall (v2): 0.8607
  Class 2 recall (v2): 0.9603
  Class 3 recall (v2): 0.7891
  Class 4 recall (v2): 0.8919
Rare-class mean recall (v2): 0.8755
